In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import datetime as dt
import os
import pycountry
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys

## Short-term business statistics

In [40]:
# Production in Construction
Produ = glob.glob('Production in Construction/*.csv')
print(Produ)

# Building Permits
Building = glob.glob('Building Permit/*.csv')
print(Building)

# Construciton Producer
Cons_Prod = glob.glob('Construction Producer/*.csv')
print(Cons_Prod)

# Labor
Labor= glob.glob('Labor/*.csv')
print(Labor)

# STS
STS = glob.glob('Structural/*.csv')
print(STS)

# GFCF
GFCF = glob.glob('GFCF/*.csv')
print(GFCF)

# GFCF ML Data - added later in project
GFCF_ML = glob.glob('GFCF-ML/*.csv')
print(GFCF_ML)

# GFCF Millions Data - added later in project
GFCF_M = glob.glob('GFCF Millions/*.csv')
print(GFCF_M)

# GVA Data and wages - added later in project
GVA = glob.glob('GVA-ML/*.csv')
print(GVA)

['Production in Construction\\civil_a.csv', 'Production in Construction\\cons_a.csv']
['Building Permit\\Building Permit Annual.csv']
['Construction Producer\\GVA.csv']
['Labor\\LaborInput Annual Employment.csv', 'Labor\\Volume of Hours.csv', 'Labor\\Wages and Salaries.csv']
['Structural\\Annual Detailed Enterprise Production Value in Euros.csv', 'Structural\\Construction by employment size class.csv', 'Structural\\Multiyear enterprise.csv', 'Structural\\Turnover statistics.csv']
['GFCF\\GFCF Construction.csv', 'GFCF\\GFCF Dwelling.csv']
['GFCF-ML\\Percent GDP.csv']
['GFCF Millions\\GFCF_million.csv']
['GVA-ML\\GVA.csv', 'GVA-ML\\GVAw_s.csv']


# Production in Construction

Will be attempting to use same functions for similar eurostat datasets, as it is more effiecient

In [41]:
# Production in Construction Extraction and Information

# Function to print stats for each file
def data_summary(file):

    df = pd.read_csv(file)
    print(f'The columns in each data {file}: \n {df.columns}\n')
    print(f'The shape of {file} is\n {df.shape}\n')
    print(f'The Info of the Data {file}:\n {df.info()}\n')
    print(f'The Summary of the Data {file}: \n {df.describe()}\n')
    print(f'Null Value sum present in {file}:\n {df.isnull().sum()}\n')
    print(f'Duplicated of {file} is\n {df.duplicated().sum()}\n')
    print(f'Unique Value Sum in {file}:\n{df.nunique()}\n')

# Function for getting full country name using pycountry
def get_country_name(country_code):
      if country_code == 'UK':
             return 'United Kingdom'
      elif country_code == 'EL':
            return 'Greece'

      try:
             country = pycountry.countries.get(alpha_2=country_code).name
             return country
      except AttributeError:
             return "Invalid country code"
      except LookupError:
             return "Invalid country code"

# Clean and transform a dataset
def clean_and_transform(file,drop,val=None):

    df = pd.read_csv(file)

    # Drop the columns that are not needed
    df.drop(drop, axis=1, inplace=True)

    # Convert Time Period to datetime format
    df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], format='%Y')
    df['TIME_PERIOD'] = df['TIME_PERIOD'].dt.year

    # Rename the columns
    if val != None:
        df = df.rename(columns={'TIME_PERIOD':'Year', 'OBS_VALUE': val})

    else:
        df = df.rename(columns={'TIME_PERIOD':'Year'})

    cleaned_df = df

    # Convert the geo to Country Name
    cleaned_df['Country'] = cleaned_df.apply(lambda row: get_country_name(row.geo), axis=1)
    cleaned_df.drop('geo', axis=1, inplace=True)

    return cleaned_df

# Transformation
concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'nace_r2', 's_adj','OBS_FLAG']

# merging all the data files into a single df
for file in Produ:

    data_summary(file)

    cleaned_df = clean_and_transform(file,val='Production Value',drop=cols_drop)
    
    if joined_df is None:
        joined_df = cleaned_df  # Set the first cleaned DataFrame as joined_df
    else:
        joined_df = joined_df.merge(cleaned_df) # Merge the cleaned DataFrame with joined_df

joined_df.to_csv('Transformed Data/Production in Construction.csv', index=False)

The columns in each data Production in Construction\civil_a.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'nace_r2', 's_adj',
       'unit', 'geo', 'TIME_PERIOD', 'OBS_VALUE 1', 'OBS_FLAG'],
      dtype='object')

The shape of Production in Construction\civil_a.csv is
 (318, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     318 non-null    object 
 1   LAST UPDATE  318 non-null    object 
 2   freq         318 non-null    object 
 3   indic_bt     318 non-null    object 
 4   nace_r2      318 non-null    object 
 5   s_adj        318 non-null    object 
 6   unit         318 non-null    object 
 7   geo          318 non-null    object 
 8   TIME_PERIOD  318 non-null    int64  
 9   OBS_VALUE 1  292 non-null    float64
 10  OBS_FLAG     69 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage:

For the Civil Engineering sector, the short-term business statistics (STS) are available for the following indicators:

The indicator is in 2015-100 indices, and the frequency of the Time Period is Annually.
The important columns for anlaysis are the following:

i. geo: Region indicator

ii. TIME_PERIOD: Time period

iii. OBS_VALUE: Observation value

iv. unit: Unit of measure

For the Construction sector, the short-term business statistics are available for the following indicators:
The indicator is in 2015-100 indices, and the frequency of the Time Period is Annually.
The important columns for anlaysis are the following:

i. geo: Region indicator   

ii. TIME_PERIOD: Time period

iii. OBS_VALUE: Observation value

iv. unit: Unit of measure

There are 25 unique countries in Civil Engineering and 27 in Construction.
minumun year period is 2010, although we have
that there are 10 missing values for OBS_VALUE in the Civil Engineering, none in Construction.



# Building Permit

In [42]:
# Building Permit Extraction and Information

concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'cpa2_1', 's_adj','OBS_FLAG']

# Transformation
for file in Building:

    data_summary(file)

    cleaned_df = clean_and_transform(file,val='Metered Squared Value',drop=cols_drop)

    if joined_df is None:
        joined_df = cleaned_df  # Set the first cleaned DataFrame as joined_df
    else:
        joined_df = joined_df.merge(cleaned_df) # Merge the cleaned DataFrame with joined_df


cleaned_df.to_csv('Transformed Data/Building Permit.csv', index=False)

The columns in each data Building Permit\Building Permit Annual.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'cpa2_1', 's_adj',
       'unit', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

The shape of Building Permit\Building Permit Annual.csv is
 (310, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     310 non-null    object 
 1   LAST UPDATE  310 non-null    object 
 2   freq         310 non-null    object 
 3   indic_bt     310 non-null    object 
 4   cpa2_1       310 non-null    object 
 5   s_adj        310 non-null    object 
 6   unit         310 non-null    object 
 7   geo          310 non-null    object 
 8   TIME_PERIOD  310 non-null    int64  
 9   OBS_VALUE    310 non-null    float64
 10  OBS_FLAG     55 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory u

## VALUE ADDED BY CONSTRUCTION in MILLION EURO

In [43]:
# Construction Producer Price Index Extraction and Information

concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'nace_r2', 'na_item','OBS_FLAG']

# Transformation
for file in Cons_Prod:

    data_summary(file)

    cleaned_df = clean_and_transform(file,val='Construction Producer Price',drop=cols_drop)

    if joined_df is None:
        joined_df = cleaned_df  # Set the first cleaned DataFrame as joined_df
    else:
        joined_df = joined_df.merge(cleaned_df) # Merge the cleaned DataFrame with joined_df

cleaned_df.to_csv('Transformed Data/GVA.csv', index=False)

The columns in each data Construction Producer\GVA.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'nace_r2', 'na_item', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

The shape of Construction Producer\GVA.csv is
 (366, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     366 non-null    object 
 1   LAST UPDATE  366 non-null    object 
 2   freq         366 non-null    object 
 3   unit         366 non-null    object 
 4   nace_r2      366 non-null    object 
 5   na_item      366 non-null    object 
 6   geo          366 non-null    object 
 7   TIME_PERIOD  366 non-null    int64  
 8   OBS_VALUE    366 non-null    float64
 9   OBS_FLAG     25 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 28.7+ KB
The Info of the Data Construction Producer\GVA.csv:
 None

The

## Labor

In [44]:
#Labor Index Extraction and Information

concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'nace_r2', 's_adj','OBS_FLAG']

# Transformation for multiple (3) data files

for file in Labor:

    data_summary(file)

    cleaned_df = clean_and_transform(file,val=None,drop=cols_drop)

    name = None

    if 'Employment' in file:
        name = "Labor_Employment.csv"
    elif 'Hours' in file:
        name = "Labor_Hours.csv"
    elif 'Wages' in file:
        name = "Labor_Wages.csv"
    else:
        print(f'Error with unexpected file :{file}')
        sys.exit(1)

    cleaned_df.to_csv(f'Transformed Data/{name}', index=False)

The columns in each data Labor\LaborInput Annual Employment.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_bt', 'nace_r2', 's_adj',
       'unit', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

The shape of Labor\LaborInput Annual Employment.csv is
 (417, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     417 non-null    object 
 1   LAST UPDATE  417 non-null    object 
 2   freq         417 non-null    object 
 3   indic_bt     417 non-null    object 
 4   nace_r2      417 non-null    object 
 5   s_adj        417 non-null    object 
 6   unit         417 non-null    object 
 7   geo          417 non-null    object 
 8   TIME_PERIOD  417 non-null    int64  
 9   OBS_VALUE    416 non-null    float64
 10  OBS_FLAG     31 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3

## Structural Business Statistics

In [45]:
concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_sb', 'nace_r2','OBS_FLAG']

# Transformation for multiple (4) data files

for file in STS:

    data_summary(file)

    cleaned_df = clean_and_transform(file,val=None,drop=cols_drop)

    name = None

    if 'Enterprise Production' in file:
        name = "STS_EnterpriseProduction.csv"
    elif 'Construction' in file:
        name = "STS_Employment.csv"
    elif 'Turnover' in file:
        name = "STS_Turnover.csv"
    elif 'Multi' in file:
        name = "STS_SubContracting.csv"
    else:
        print(f'Error with unexpected file :{file}')
        sys.exit(1)

    cleaned_df.to_csv(f'Transformed Data/{name}', index=False)

The columns in each data Structural\Annual Detailed Enterprise Production Value in Euros.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'nace_r2', 'indic_sb', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

The shape of Structural\Annual Detailed Enterprise Production Value in Euros.csv is
 (265, 9)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     265 non-null    object 
 1   LAST UPDATE  265 non-null    object 
 2   freq         265 non-null    object 
 3   nace_r2      265 non-null    object 
 4   indic_sb     265 non-null    object 
 5   geo          265 non-null    object 
 6   TIME_PERIOD  265 non-null    int64  
 7   OBS_VALUE    264 non-null    float64
 8   OBS_FLAG     26 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 18.8+ KB
The Info of the Data Structural\Annua

## Gross Fixed Capital Formation Extraction and Information

In [46]:
concatenated_df = pd.DataFrame()
joined_df = None

cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'asset10','OBS_FLAG']

# Transformation for multiple (4) data files

for file in GFCF:

    data_summary(file)



data_frames = [clean_and_transform(file,val=None,drop=cols_drop) for file in GFCF]
df1= data_frames[0]
df2 = data_frames[1]

df1 = df1.rename(columns={'OBS_VALUE':'Construction Value'})
df2 = df2.rename(columns={'OBS_VALUE':'Dwellings Value'})

joined = df1.merge(df2)

print(joined)

joined.to_csv('Transformed Data/GFCF.csv', index=False)

The columns in each data GFCF\GFCF Construction.csv: 
 Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'asset10', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

The shape of GFCF\GFCF Construction.csv is
 (220, 9)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     220 non-null    object 
 1   LAST UPDATE  220 non-null    object 
 2   freq         220 non-null    object 
 3   unit         220 non-null    object 
 4   asset10      220 non-null    object 
 5   geo          220 non-null    object 
 6   TIME_PERIOD  220 non-null    int64  
 7   OBS_VALUE    220 non-null    float64
 8   OBS_FLAG     30 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 15.6+ KB
The Info of the Data GFCF\GFCF Construction.csv:
 None

The Summary of the Data GFCF\GFCF Construction.csv: 
        TIME_P

## GFCF % GDP Data for ML

In [47]:
cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'asset10','OBS_FLAG','unit']

data_frames = [clean_and_transform(file,val=None,drop=cols_drop) for file in GFCF_ML]
df= data_frames[0]

df = df.rename(columns={'OBS_VALUE':'Percent GDP'})

df.to_csv('Transformed Data/GFCF-ML.csv', index=False)

df.head()

,Year,Percent GDP,Country
0,1996,6.9,Albania
1,1997,6.4,Albania
2,1998,7.1,Albania
3,1999,7.3,Albania
4,2000,10.3,Albania


## GFCF Millions Data for ML

In [48]:
cols_drop = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'asset10', 'OBS_FLAG']

data_frames = [clean_and_transform(file,val='Value',drop=cols_drop) for file in GFCF_M]
df= data_frames[0]

df.to_csv('Transformed Data/GFCF-M.csv', index=False)

df.head()

,Year,Value,Country
0,1995,25965.5,Austria
1,1996,26269.2,Austria
2,1997,25887.2,Austria
3,1998,26332.7,Austria
4,1999,26697.9,Austria


## GVA Data for ML

In [50]:
data_frames = []

for file in GVA:

    f = pd.read_csv(file)
    print(file)
    print(f.columns)

    if 'w_s' in file:
        cols_drop = ['DATAFLOW', 'LAST UPDATE','freq', 'nace_r2', 'unit', 'na_item', 'OBS_FLAG']
        val = 'Wages'
    else:
        cols_drop = ['DATAFLOW', 'LAST UPDATE','freq', 'nace_r2', 'unit', 'na_item', 'OBS_FLAG']
        val = 'GVA'

    df =  clean_and_transform(file,val=val,drop=cols_drop)

    data_frames.append(df)


gvas = data_frames[0].merge(data_frames[1])

gvas.to_csv('Transformed Data/GVA-ML.csv', index=False)

gvas.head()

GVA-ML\GVA.csv
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'nace_r2', 'na_item', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')
GVA-ML\GVAw_s.csv
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'nace_r2', 'na_item', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')


,Year,GVA,Country,Wages
0,2000,14308.8,Austria,6878.6
1,2001,14000.3,Austria,6749.1
2,2002,14131.4,Austria,6588.6
3,2003,15085.1,Austria,6598.2
4,2004,15602.3,Austria,6695.5
